In [5]:
import numpy as np
import os
import pandas as pd 
import sys
import scanpy as sc

In [8]:
sys.path.insert(1, '/Users/jinmr2/Dropbox/Code/ToppCell-Python/')
import ToppCell_Python as tp

In [6]:
adata = sc.read("/Users/jinmr2/Dropbox/Code/data/batch2_all_normalized_filtered.h5ad")

In [39]:
shred = tp.Shred(adata = adata,
            shred_plan = ["stim", "cell", "stim+cell|stim"],
            bin_group = ["stim", "cell"],
            order_bins = None,
            order_modules = None,
            method = "wilcoxon",
            output_dir = "/Users/jinmr2/Dropbox/Code/data/toppcell_test/")

In [10]:
shred.do_shredplan()

/Users/jinmr2/opt/miniconda3/envs/aipy/lib/python3.7/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'bin_group' as categorical
/Users/jinmr2/opt/miniconda3/envs/aipy/lib/python3.7/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'bin_id' as categorical
/Users/jinmr2/opt/miniconda3/envs/aipy/lib/python3.7/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated

In [11]:
shred.create_heatmap_matrix()

In [37]:
def createGCT_file(shred, type_bin):
    """
    create GCT file based on both heatmap matrix and bin / module metadata
    """
    # initialization
    if type_bin == "bin":
        bin_meta = shred.bin_metadata.copy()
        heatmap_matrix = shred.heatmap_matrix.copy()
    elif type_bin == "superbin":
        bin_meta = shred.superbin_metadata.copy()
        heatmap_matrix = shred.heatmap_matrix_super.copy()
    else:
        raise Exception("type should be either bin or superbin")
    df_module = shred.shred_modules_df_2.copy()

    # format table
    bin_meta = bin_meta.loc[list(heatmap_matrix.columns), :]
    bin_meta["bin_id"] = bin_meta.index.values

    df_module.reset_index(level = 0, inplace = True)
    df_module.rename({"names": "Genes"}, axis = 1, inplace = True)

    df_new_heatmap = pd.DataFrame(data = heatmap_matrix.values,
                                    index = pd.MultiIndex.from_frame(df_module),
                                    columns = pd.MultiIndex.from_frame(bin_meta))

    gct_output_name = "heatmap_matrix_GCT.gct" if type_bin == "bin" else "heatmap_matrix_superbin_GCT.gct"
    df_new_heatmap.to_csv(shred.output_folder + gct_output_name, sep = "\t")
    
    # reformat the table into GCT3 type.
    second_line = str(heatmap_matrix.shape[0]) + "\t" + str(heatmap_matrix.shape[1]) + "\t" + str(df_module.shape[1]-1) + "\t" + str(bin_meta.shape[1]-1) + "\n"
    
    third_line = ""
    for i, col in enumerate(df_module.columns):
        if i != (df_module.shape[1] - 1):
            third_line += (col + "\t")
        else:
            third_line += col
    third_line_bins = "\t" + "\t".join(list(bin_meta.index.values)) + "\n"
    third_line += third_line_bins
    
    with open(shred.output_folder + gct_output_name, "r") as f:
        lines = f.readlines()
    with open(shred.output_folder + gct_output_name, "w") as f:
        f.write("#1.3\n")
        f.write(second_line)
        f.write(third_line)
        for line in lines:
            if not (line.startswith("Genes") or line.startswith("bin_id")):
                f.write(line)

In [38]:
createGCT_file(shred, "superbin")

### test for batch run

In [40]:
shred = tp.Shred(adata = adata,
            shred_plan = ["stim", "cell", "stim+cell|stim"],
            bin_group = ["stim", "cell"],
            order_bins = None,
            order_modules = None,
            method = "wilcoxon",
            output_dir = "/Users/jinmr2/Dropbox/Code/data/toppcell_test/")

In [41]:
shred.toppcell_batchRun(enrich_categories = ["GeneOntologyCellularComponent"])

/Users/jinmr2/opt/miniconda3/envs/aipy/lib/python3.7/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'bin_group' as categorical
/Users/jinmr2/opt/miniconda3/envs/aipy/lib/python3.7/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'bin_id' as categorical
/Users/jinmr2/opt/miniconda3/envs/aipy/lib/python3.7/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated

TypeError: draw_heatmap() takes 1 positional argument but 2 were given